### Given p[i,t] and b[i,j],  Estimate delta[i]
#### input:
p[i,t] is ndarray,
the delay ratio of airport i at time t;

b[i,j] is the normalize the infection rate from j to i,
sum(b[i,:]) = 1
#### output
ndarray delta[i]

In [1]:
def estimate_delta_v1( p , b ):
    # 2019-7-12  Jian Wang

    # p[i,t]   
    # b[i,j]
    
    n = len( p )  # numbers of airport
    m = len( p[0] ) # numbers of time
    
    if ( len(b)!= n ) or ( len(b[0])!= n ):
        print("ERROR, airport number not consistent")
        return None
    
    k = np.ndarray( shape=(n,m) )
    
    for i in range(n):
        for t in range(m):
            k[i,t] = sum( p[j,t] * (1-p[i,t]) * b[i,j] for j in range(n) )
            
    # p[i,t+1] - p[i,t] = - delta[i] * p[i,t] + k[i,t]
    # t = 0,1,2,...,(m-2)
    out = []
    for i in range(n):
        Yi = np.asarray( [ p[i,t] + k[i,t] - p[i,t+1] for t in range(m-1) ])
        Xi = np.asarray( [ p[i,t] for t in range(m-1) ])
        delta , c = np.linalg.lstsq(np.vstack([Xi, np.ones(len(Xi))]).T, Yi)[0]
        out.append( delta )
    return np.asarray(out)

### Given p[i,t], Estimate delta[i] and b[i,j]
#### input:
p[i,t] is ndarray,
the delay ratio of airport i at time t;

#### output
ndarray delta[i]  
b[i,j] is the normalize the infection rate from j to i,  
sum(b[i,:]) = 1

### Loss function - Mean square error

#### input:
p[i,t] is ndarray,
the delay ratio of airport i at time t;

b[i,j] is the normalize the infection rate from j to i,
sum(b[i,:]) = 1

delta[i] ndarray 
#### how is $\hat{p}[i,t] $ caculated?

Assume $\Delta t = 1 \text{day} $ in such expansion:

$$ \frac{p[i,t+\Delta t] - p[i,t] }{ \Delta t } =   - \delta[i] p[i,t] + k[i,t]
$$

Then we can predict the value of p, based on yesterday's information from all airports.

$$ \hat{p}[i,t] = p[i,t-1] - \delta[i] p[i,t-1] + k[i,t-1]
$$

here $k[i,t] = \sum_j b[i,j] p[j,t] (1-p[i,t])$
#### output:
Mean square error |phat-p|^2 over time and airports
$$
\text{output} = \frac{ \sum_{i} \sum_{t} | \hat{p}[i,t] - p[i,t] |^2  }{\sum_{i} \sum_{t} 1}
$$

In [2]:
def loss_v1( p , b , delta ):
    # 2019-7-12  Jian Wang
    
    # p[i,t]
    # b[i,j]
    # delta[i]
    
    n = len( p )  # numbers of airport
    m = len( p[0] ) # numbers of time
    
    if ( len(b)!= n ) or ( len(b[0])!= n ) or ( len(delta) != n):
        print("ERROR, airport number not consistent")
        return None
    
    k = np.ndarray( shape=(n,m) )
    
    for i in range(n):
        for t in range(m):
            k[i,t] = sum( p[j,t] * (1-p[i,t]) * b[i,j] for j in range(n) )
            
    # phat[i,t+1] - p[i,t+1] 
            
    # phat[i,t+1]  = p[i,t] - delta[i] * p[i,t] + k[i,t]
    # t = 0,1,2,...,(m-2)
    
    
    out = 0.0
    for i in range(n):
        for t in range(1,m):
            phat = p[i,t-1] - delta[i] * p[i,t-1] + k[i,t-1]
            out = out + (phat - p[i,t] )**2
    out = out / (n*(m-1))
    return out